<a href="https://colab.research.google.com/github/ericr6/platform_analysis/blob/main/hw_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Hello World Monitoring

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import random

# @title import stuffs
import requests
import json
import csv
from pprint import pprint
import pandas as pd
from datetime import datetime



# Assuming your DataFrame is named 'df'
# Convert DATE column to datetime if it's not already
# Replace 'specific_workerpool_id' with the actual ID of the worker pool you're interested in

def taskperday(df,workerpool_id, wpname):

    # Filter the DataFrame further for the specific worker pool
    df_taskday = df[df['WORKERPOOL ID'] == workerpool_id]
    len(df_taskday)

    df_taskday['DATE'] = pd.to_datetime(df_taskday['DATE'])
    # Define the date interval
    start_date = df_taskday['DATE'].min().strftime("%Y-%m-%d")
    end_date = df_taskday['DATE'].max().strftime("%Y-%m-%d")

    # Extract month from the DATE column
    df_taskday['Day'] = df_taskday['DATE'].dt.to_period('d')

    # Group by Month and STATUS, count occurrences
    status_counts = df_taskday.groupby(['Day', 'STATUS']).size().unstack(fill_value=0)

    # Plot
    status_counts.plot(kind='bar', stacked=True, figsize=(20, 8))

    plt.title('Tasks per day \n (Workerpool: {}) \n {} Start Date: {} | End Date: {}'.format(workerpool_id, wpname, start_date, end_date))
    plt.xlabel('Month')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.legend(title='Status')

    #Upscale resolution
    plt.figure(dpi=300)
    plt.show()


# Assuming df is your DataFrame containing the dApps data
# Filter the DataFrame based on the 'WORKERPOOL ID'
#workerpool_filter = ('0x0e7bc972c99187c191a17f3cae4a2711a4188c3f','0xdb214a4a444d176e22030be1ed89da1b029320f2')
#duration=(1,7,30);


def taskplot(df,workerpool_filter, duration, wp_name):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=duration)
    date_filtered_df = df[(df['DATE'] >= start_date) & (df['DATE'] <= end_date)]
    filtered_df = date_filtered_df[date_filtered_df['WORKERPOOL ID'] == workerpool_filter]

    # Group by app name and date to count the number of uses of each dApp on each date
    app_usage = filtered_df.groupby(['APP NAME', pd.Grouper(key='DATE', freq='D')]).size().unstack(fill_value=0)

    # Get unique app names for y-axis ticks
    dapp_names = app_usage.index

    # Calculate figsize based on the number of dApps
    fig_height = max(1, len(dapp_names) * 0.5)  # Minimum height of 6 inches
    plt.figure(figsize=(25, fig_height))

    # Plot usage events for each dApp
    for i, app in enumerate(dapp_names):
        usage_dates = filtered_df[filtered_df['APP NAME'] == app]['DATE']
        plt.plot(usage_dates, [i] * len(usage_dates), marker='o', linestyle='', markersize=5, label=app)

    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    plt.title('Usage of dApps Over Time \n Workerpool: {} \n {} \n Last {} day(s)  [{} -> {}]'.format(wp_name,workerpool_filter, str(duration), start_date_str, end_date_str))

    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.ylabel('dApp')
    plt.yticks(range(len(dapp_names)), dapp_names)  # Set y-axis ticks to be the unique app names
    # Extend y-axis plot
    plt.ylim(-0.5, len(dapp_names) - 0.5)

  #plt.legend()  # Add legend to show dApp names

    plt.grid(axis='y')
    plt.show()    


# Function to calculate the percentage of successful tasks
def calculate_success_percentage(total_tasks, successful_tasks):
    if total_tasks == 0:
        return 0
    return (successful_tasks / total_tasks) * 100

# Function to plot a pie chart with percentage and value
def plot_pie_chart(ax, successful_tasks, total_tasks, title):
    success_percentage = calculate_success_percentage(total_tasks, successful_tasks)
    failed_tasks = total_tasks - successful_tasks

    ax.pie([successful_tasks, failed_tasks], labels=[f'Successful Tasks ({successful_tasks})', f'Failed Tasks ({failed_tasks})'], autopct='%1.1f%%', startangle=140,colors=['lightgreen', 'lightcoral'])
    ax.title(title + f' ({success_percentage:.2f}% success)', fontsize=12, fontweight='bold')
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    ax.legend(loc='upper right', fontsize=10)

# Define the date interval
date_begin = df_tasks_30d['DATE'].min().strftime("%Y-%m-%d")
date_end = df_tasks_30d['DATE'].max().strftime("%Y-%m-%d")

# Assuming 'df_tasks_30d' is your DataFrame with task data



def successrate(dftmp, workerpool_filter, duration, wp_name ):
  # Calculate the date range for each time interval
  # Replace 'specific_workerpool_id' with the actual ID of the worker pool you're interested in
  
  end_date = datetime.now()
  start_date = end_date - timedelta(days=duration)

  # Filter the DataFrame for each time interval
  df_datefiltered = dftmp[(dftmp['DATE'] >= start_date) & (dftmp['DATE'] <= end_date)]

  # Filter the DataFrame further for the specific worker pool
  df_tmp = df_datefiltered[df_datefiltered['WORKERPOOL ID'] == workerpool_filter]


  # Calculate the total number of tasks and successful tasks for each time interval
  total_tasks = len(df_tmp)
  successful_tasks = len(df_tmp[df_tmp['STATUS'] == 'COMPLETED'])
  
  # Create a figure with subplots
#  fig, axs = plt.subplots(1, 4, figsize=(20, 8))
  # Plot pie charts for each time interval

  start_date_str = start_date.strftime("%Y-%m-%d")
  end_date_str = end_date.strftime("%Y-%m-%d")
  title=('Success Rate on {} \n Workerpool: \n {} \n Last {} day(s) \n [{} -> {}]'.format(wp_name,workerpool_prod, str(duration), start_date_str, end_date_str))
 
  plot_pie_chart( plt, successful_tasks, total_tasks, title)
  
  # Adjust layout and show plot
  plt.tight_layout()
  plt.show()

def success_repartition(df_tmp, workerpool, duration, wp_name):

    # Filter the DataFrame for the last day
    end_date = datetime.now()
    start_date = end_date - timedelta(days=duration)
    df_datefiltered = df_tmp[(df_tmp['DATE'] >= start_date) & (df_tmp['DATE'] <= end_date)]
    # Filter the DataFrame further for the specific worker pool
    df_wpfiltered = df_datefiltered[df_datefiltered['WORKERPOOL ID'] == workerpool]

    # Create a dictionary to map DApp names to colors and marker types
    dapp_info = {}
    for i, dapp in enumerate(df_wpfiltered['APP NAME'].unique()):
        dapp_info[dapp] = {
            'color': plt.cm.tab20(i/len(df_wpfiltered['APP NAME'].unique())),
            'marker': 'o'  # Use the same marker type for each DApp
        }

    # Plot successful and unsuccessful points using the same marker type per DApp
    plt.figure(figsize=(20, 6))
    for dapp, info in dapp_info.items():
        dapp_df = df_wpfiltered[df_wpfiltered['APP NAME'] == dapp]
        success_mask = dapp_df['STATUS'] == 'COMPLETED'

        # Generate random y-coordinates for successful points in the range [0.1, 1]
        random_success_y = np.random.uniform(low=0.1, high=1, size=np.sum(success_mask))
        # Generate random y-coordinates for unsuccessful points in the range [-1, -0.1]
        random_failure_y = np.random.uniform(low=-1, high=-0.1, size=np.sum(~success_mask))

        # Plot successful points
        plt.scatter(dapp_df['DATE'][success_mask], random_success_y, color=info['color'], label=f'{dapp}', marker=info['marker'])
        # Plot unsuccessful points
        plt.scatter(dapp_df['DATE'][~success_mask], random_failure_y, color=info['color'], marker=info['marker'])

    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    title=('Tasks Success/fail distribution on \n {}  \n Workerpool: \n {} \n Last {} day(s) \n [{} -> {}]'.format(wp_name, workerpool, str(duration), start_date_str, end_date_str))
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Task Status')
    plt.yticks([-1, 0, 1], ['Failed', '', 'Success'])

    # Create a single legend for both success and failure
    plt.legend(title='DApp', loc='center left', bbox_to_anchor=(1, 0.5))

    plt.grid(True)
    plt.tight_layout()

    plt.show()

def successrate_app(df_tmp, workerpool, wp_name):
    # Filter the DataFrame to include only tasks associated with the specified workerpool
    workerpool_df = df_tmp[df_tmp['WORKERPOOL ID'] == workerpool]

    # Define the date interval
    date_begin = df_tmp['DATE'].min().strftime("%Y-%m-%d")
    date_end = df_tmp['DATE'].max().strftime("%Y-%m-%d")

    # Calculate the success ratio for each DApp within the specified workerpool
    dapp_data = {}
    for dapp, group in workerpool_df.groupby('APP NAME'):
        total_count = group.shape[0]
        success_count = group[group['STATUS'] == 'COMPLETED'].shape[0]
        ratio = success_count / total_count if total_count != 0 else 0
        dapp_data[dapp] = {'ratio': ratio, 'success_count': success_count, 'total_count': total_count}

    # Sort the dapp_data dictionary by the total_count of tasks
    sorted_dapp_data = dict(sorted(dapp_data.items(), key=lambda item: item[1]['total_count'], reverse=False))

    # Create a bar plot
    fig_height = max(1, len(sorted_dapp_data) * 0.3)  # Calculate figsize based on the number of dApps
    plt.figure(figsize=(25, fig_height))

    plt.barh(list(sorted_dapp_data.keys()), [d['ratio'] for d in sorted_dapp_data.values()], color='lightgreen', label='Success Ratio')
    plt.xlabel('Success Ratio')
    plt.ylabel('DApp')
    plt.title(f'Success Ratio of Tasks for Each DApp \n ({date_begin} to {date_end}) \n Workerpool {wp_name} : {workerpool}')
    plt.legend()

    # Add annotations for success ratio and total task count
    for i, (dapp, data) in enumerate(sorted_dapp_data.items()):
        plt.text(data['ratio'], i, f'{data["ratio"]:.2f} {data["success_count"]} out of {data["total_count"]} tasks', verticalalignment='center')

    plt.tight_layout()
    plt.show()

# Requester Activity Analysis for a defined app in last days

def req_plot(df,workerpool_filter, duration, wp_name, app):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=duration)
    date_filtered_df = df[(df['DATE'] >= start_date) & (df['DATE'] <= end_date)]
    filtered_df = date_filtered_df[date_filtered_df['WORKERPOOL ID'] == workerpool_filter]
    filteredapp_df = filtered_df[filtered_df['APP NAME'] == app]


    # Group by app name and date to count the number of uses of each dApp on each date
    # req_usage = filteredapp_df.groupby(['REQUESTER ID', pd.Grouper(key='DATE', freq='D')]).size().unstack(fill_value=0)
    # Define colors for different statuses
    status_colors = {'COMPLETED': 'green', 'ACTIVE': 'red'}
    address_counts = filteredapp_df['REQUESTER ID'].value_counts()
 

    # Create a scatter plot with conditional coloring based on the status
    plt.figure(figsize=(20, 14))
    annotated_ids = set()  # Initialize a set to keep track of annotated IDs
    for status, color in status_colors.items():
        status_df = filteredapp_df[filteredapp_df['STATUS'] == status]
        plt.scatter(status_df['DATE'], status_df['REQUESTER ID'], marker='o', color=color, label=status)

        # Annotate points with the number of points for each address
        for idx, row in status_df.iterrows():
            if row['REQUESTER ID'] not in annotated_ids:  # Check if ID has not been annotated yet
                plt.annotate(str(address_counts[row['REQUESTER ID']]), (row['DATE'], row['REQUESTER ID']), textcoords="offset points", xytext=(-10,0), ha='right')
                annotated_ids.add(row['REQUESTER ID'])  # Add ID to annotated set

    # Set plot title and labels
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    plt.title('Scatter Plot of Requesters for \n DApp {} \n Workerpool: {} ; {}\n Last {} day(s)    [{} -> {}]'.format(app, wp_name, workerpool_filter, duration, start_date_str, end_date_str))

    plt.xlabel('Date')
    plt.ylabel('Requester')

    # Rotate x-axis labels for better readability if needed
    plt.xticks(rotation=45)

    # Show plot
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Requester usage of specific app and workerpool in last days

def req_count(df,workerpool_filter, duration, wp_name, app):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=duration)
    date_filtered_df = df[(df['DATE'] >= start_date) & (df['DATE'] <= end_date)]
    filtered_df = date_filtered_df[date_filtered_df['WORKERPOOL ID'] == workerpool_filter]
    filteredapp_df = filtered_df[filtered_df['APP NAME'] == app]


    # Group by 'APP NAME' and 'STATUS' and count occurrences
    app_counts = filteredapp_df.groupby(['REQUESTER ID', 'STATUS']).size()

    # Reset index to make 'APP NAME' and 'STATUS' as columns
    app_counts_df = app_counts.reset_index(name='Number of Occurrences')

    # Pivot the DataFrame to have 'STATUS' as columns
    app_counts_pivot = app_counts_df.pivot(index='REQUESTER ID', columns='STATUS', values='Number of Occurrences')

    # Replace NaN values with zero
    app_counts_pivot.fillna(0, inplace=True)

    # Convert the DataFrame to use integer data type
    app_counts_pivot = app_counts_pivot.astype(int)

    # Add 'COMPLETED' column as sum of 'COMPLETED' and 'FAILED' columns
    app_counts_pivot['COMPLETED'] = app_counts_pivot['COMPLETED']

    # Sort the DataFrame based on 'COMPLETED' column
    app_counts_pivot_sorted = app_counts_pivot.sort_values(by='COMPLETED', ascending=False)

    # Print the DataFrame completely
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    print('Requester Activity for DApp {} on Workerpool: {} ; {}  Last {} day(s)    [{} -> {}]'.format(app, wp_name, workerpool_filter, duration, start_date_str, end_date_str))
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # Display all rows and columns
        print(app_counts_pivot_sorted)

def req_count_with_detail(df, workerpool_filter, duration, wp_name, app):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=duration)
    
    # Filter data by date and workerpool
    date_filtered_df = df[(df['DATE'] >= start_date) & (df['DATE'] <= end_date)]
    filtered_df = date_filtered_df[date_filtered_df['WORKERPOOL ID'] == workerpool_filter]
    filteredapp_df = filtered_df[filtered_df['APP NAME'] == app]
    
    # First, get APP MULTIADDR analysis
    app_multiaddr_analysis = filteredapp_df.groupby('APP MULTIADDR')['REQUESTER ID'].unique()
    
    # Group by 'APP NAME' and 'STATUS' and count occurrences
    app_counts = filteredapp_df.groupby(['REQUESTER ID', 'STATUS']).size()
    
    # Reset index to make 'APP NAME' and 'STATUS' as columns
    app_counts_df = app_counts.reset_index(name='Number of Occurrences')
    
    # Pivot the DataFrame to have 'STATUS' as columns
    app_counts_pivot = app_counts_df.pivot(index='REQUESTER ID', columns='STATUS', values='Number of Occurrences')
    
    # Replace NaN values with zero
    app_counts_pivot.fillna(0, inplace=True)
    
    # Convert the DataFrame to use integer data type
    app_counts_pivot = app_counts_pivot.astype(int)
    
    # Add 'COMPLETED' column as sum of 'COMPLETED' and 'FAILED' columns
    app_counts_pivot['COMPLETED'] = app_counts_pivot['COMPLETED']
    
    # Add APP MULTIADDR column
    requester_app_multiaddr = filteredapp_df.groupby('REQUESTER ID')['APP MULTIADDR'].first()
    app_counts_pivot['APP MULTIADDR'] = app_counts_pivot.index.map(requester_app_multiaddr)
    
    # Sort the DataFrame based on 'COMPLETED' column
    app_counts_pivot_sorted = app_counts_pivot.sort_values(by='COMPLETED', ascending=False)
    
    # Reorder columns to put APP MULTIADDR first
    cols = app_counts_pivot_sorted.columns.tolist()
    final_cols = ['APP MULTIADDR'] + [col for col in cols if col != 'APP MULTIADDR']
    app_counts_pivot_sorted = app_counts_pivot_sorted[final_cols]
    
    # Print the results
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    print('Requester Activity for DApp {} on Workerpool: {} ; {}  Last {} day(s)    [{} -> {}]'.format(
        app, wp_name, workerpool_filter, duration, start_date_str, end_date_str))
    
    # Print APP MULTIADDR analysis
    print("\nAPP MULTIADDR Analysis:")
    print("-" * 50)
    for multiaddr, requesters in app_multiaddr_analysis.items():
        print(f"\nAPP MULTIADDR: {multiaddr}")
        print("Associated Requesters:")
        for requester in requesters:
            print(f"  - {requester}")
    print("\n" + "-" * 50 + "\n")
    
    # Print the main activity table
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print("Detailed Activity by Requester:")
        print(app_counts_pivot_sorted)

    print('Requester Activity for DApp {} on Workerpool: {} ; {}  Last {} day(s)    [{} -> {}]'.format(app, wp_name, workerpool_filter, duration, start_date_str, end_date_str))


#print(filename)

# @title Common Functions
# Function to convert Hexa String to Text
def hex_to_string(value):
    if not value:
        return value

    bytes_from_hex = bytes.fromhex(value[2:])
    return bytes_from_hex.decode('utf-8')

def filter_by_time_interval(data_list, date_begin, date_end):
    # Convert date_begin and date_end strings to datetime objects
    #date_begin = datetime.strptime(date_begin, '%Y-%m-%d %H:%M:%S')
    #date_end = datetime.strptime(date_end, '%Y-%m-%d %H:%M:%S')

    # Filter data_list based on timestamp falling within the specified time interval
    filtered_data = [row for row in data_list if date_begin <= datetime.utcfromtimestamp(int(row[5])) <= date_end]
    return filtered_data



def get_api_data_iexec_interval(url_query, query, global_skip, date_begin=None, date_end=None):
    total_data = []
    are_data = True
    detected = False
    skip = 0
    while are_data and skip < 200000 or not detected:

        # Construct the query with skip parameter and optional date filters
        query_iter = query.replace("skip_param", str(global_skip + skip))

        options = {
            'headers': {'Content-Type': 'application/json'},
            'data': '{"query": "' + query_iter + '"}',
        }

        response = requests.post(url_query, **options)

        # Check if the response contains valid JSON data
        response.raise_for_status()
        data = response.json()

        array_data = data.get('data', {}).get('taskInitializes', [])

        grouped = [
            [
                e['task']['id'],
                e['task']['deal']['app']['name'],
                hex_to_string(e['task']['deal']['app']['multiaddr']),
                e['task']['deal']['tag'],
                e['task']['status'],
                #pd.to_datetime(int(e['task']['timestamp']) * 1000000).strftime('%Y-%m-%d %H:%M:%S'),
                e['task']['timestamp'],
                e['task']['deal']['workerpool']['id'],
                e['task']['deal']['requester']['id']
            ]
            for e in array_data
        ]

        grouped = filter_by_time_interval(grouped, date_begin, date_end)

        total_data.extend(grouped)
        skip += 500
        _are_data = len(grouped) > 0
        if _are_data > 0 and are_data:
 #           print ("detected is true")
            detected = True
        are_data = len(grouped) > 0
        print("i= " + str(global_skip + skip) + " are_data: " + str(are_data) + "_are_data: " + str(_are_data) + " grouped size " +  str(len(grouped)) + " array_data size: " + str(len(array_data))  )
    return total_data



# Actualize data
url_query_bellecour = 'https://thegraph.bellecour.iex.ec/subgraphs/name/bellecour/poco-v5'
query = '{\
                taskInitializes(first:500, skip: skip_param, orderBy: timestamp, orderDirection: desc){\
                  task{\
                    id,\
                    deal{\
                      requester {\
                        id\
                      }\
                      app{\
                        name\
                        multiaddr\
                      }\
                      dataset{\
                        name\
                      }\
                      workerpool{\
                        id\
                         }\
                      tag\
                    }\
                    status,\
                    timestamp\
                  }\
                }\
              }'
# @title check data set structure


# @title Get latest data ; Select N Days

date_end = datetime.now()
date_begin = date_end - timedelta(days=30)

print ("Period to retrieve:", date_begin, date_end)

api_data = get_api_data_iexec_interval(url_query_bellecour, query, 0, date_begin, date_end)
df_tasks_30d = pd.DataFrame(api_data, columns=["TASK_ID","APP NAME", "APP MULTIADDR", "TAG", "STATUS", "DATE", "WORKERPOOL ID", "REQUESTER ID"])
df_tasks_30d["DATE"] = [datetime.utcfromtimestamp(int(x)) for x in df_tasks_30d["DATE"]]

print("Period observed", df_tasks_30d["DATE"].min(), df_tasks_30d["DATE"].max())
print("dataset length: " + str(df_tasks_30d.shape[0]))

df_tasks_30d.dtypes

# @title check dataset header

df_tasks_30d


req_plot(df_tasks_30d, v8_learn_debug, 40, "V8_learn_debug", "iexec-hello-world-iapp-0.0.1")

req_count_with_detail(df_tasks_30d, v8_learn_debug, 40, "V8_learn_debug", "iexec-hello-world-iapp-0.0.1")
